#Probing the surface of the Villin headpiece protein using an Ammonium ion

This is an example of evaluating a pseudotrajectory created with molgri using this package.

In [1]:
import sys
sys.path.append('../') #workaround, need to add the module to a folder that is part of path?
import numpy as np
from tqdm import tqdm
import nglview as nv
import MDAnalysis as mda

import modelledtrajectory as mtra


First we define the files needed for the calculations. The molgri pseudotrajectory has already been made and just needs to be loaded. For the probe we load an SDF file so the proper parameters can be found using the Amber general forcefield GAFF. 



In [2]:
fVillin = 'villin.pdb'
fNH4 = 'Structure_NH4.sdf'
fPT1 = 'villin_NH4_o_ico_256_b_zero_1_t_1422031497.gro'
fPT2 = 'villin_NH4_o_ico_256_b_zero_1_t_1422031497.xtc'

mt = mtra.ModelledTrajectory(fVillin, fNH4, fPT1, fPT2, 
                             forces=['amber/protein.ff14SB.xml', 'implicit/gbn2.xml'], 
                             nonbondedMethod='CutoffPeriodic', nonbondedCutoff = 2)


/home/mdglasius/anaconda3/lib/python3.9/site-packages/MDAnalysis/lib/mdamath.py:293: RuntimeWarning: invalid value encountered in double_scalars
  alpha = np.rad2deg(np.arccos(np.dot(y, z) / (ly * lz)))
/home/mdglasius/anaconda3/lib/python3.9/site-packages/MDAnalysis/lib/mdamath.py:294: RuntimeWarning: invalid value encountered in double_scalars
  beta = np.rad2deg(np.arccos(np.dot(x, z) / (lx * lz)))
/home/mdglasius/anaconda3/lib/python3.9/site-packages/MDAnalysis/lib/mdamath.py:295: RuntimeWarning: invalid value encountered in double_scalars
  gamma = np.rad2deg(np.arccos(np.dot(x, y) / (lx * ly)))


The other parameters specify the parameter files for the forcefield that openMM should use to set up the simulation box, and settings for the nonbonded interactions that are also passed to the openMM system. We can now evaluate properties from the simulation box at each frame in the pseudotrajectory. For example, if we would want to know the potential energy at each frame we can loop over the entire trajectory:

In [3]:
potentials = np.zeros(len(mt))
for i in tqdm(mt.frames()):
    potentials[i] = mt.getPotentialEnergy()

  0%|                                                                                                                                                                 | 0/2048 [00:00<?, ?it/s]/home/mdglasius/anaconda3/lib/python3.9/site-packages/MDAnalysis/lib/mdamath.py:293: RuntimeWarning: invalid value encountered in double_scalars
  alpha = np.rad2deg(np.arccos(np.dot(y, z) / (ly * lz)))
/home/mdglasius/anaconda3/lib/python3.9/site-packages/MDAnalysis/lib/mdamath.py:294: RuntimeWarning: invalid value encountered in double_scalars
  beta = np.rad2deg(np.arccos(np.dot(x, z) / (lx * lz)))
/home/mdglasius/anaconda3/lib/python3.9/site-packages/MDAnalysis/lib/mdamath.py:295: RuntimeWarning: invalid value encountered in double_scalars
  gamma = np.rad2deg(np.arccos(np.dot(x, y) / (lx * ly)))
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2048/2048 [02:14<00:00, 15.18it/s]


Here the mt.frames() function returns an iterator that loops over the entire trajectory (the tqdm function is only used to provide a progress bar since this may take a few minutes). For each iteration the simulation box coordinates are set to those in the current frame of the trajectory. mt.getPotentialEnergy() then returns the current potential energy of the system. By using this iterator it is possible to take different steps and or readings per frame without looping over the entire trajectory for each one. 

Lets say we now want to do some measurements after a short minimization. Since we don't want to move to far from the initial gridpoint in the pseudotrajectory we would like to constrain the position of the protein backbone and the ammonium probe. We can start by selecting the ID's for the atoms that need to be restrained using MDAnalysis selection language:

In [4]:
probe = mt.selectIDs('type N and not protein')
backbone = mt.selectIDs('backbone')

This selects the nitrogen in the probe and all the atoms that are considered backbone in the protein. In order to constrain these we can call the following function:

In [5]:
mt.constrainAtoms(probe + backbone)

setting the masses of the selected atoms to 0 in the openMM simulation, causing their locations to become fixed. Note that setting these masses affects all frames of the trajectory, not just the current one. Since performing a minimization for each frame of the trajectory can take a while, we will instead select a number of the highest energy frames (that might need some minimization due to the probe and protein colliding) from the earlier list and see if the minimization affects those frames.

In [6]:
numFrames = 10
highPotentialFrames = np.argsort(potentials)[-1:-numFrames-1:-1]
print(highPotentialFrames)

[2008 1809 1608  992 1843 1585    3  161 1617 1536]


the mt.frames() function can be called with a list of integers as an argument. The resulting iterator will, instead of all frames, loop only over those specified in the argument, in the order they are specified in. Now we will perform the minimization for these frames, making sure to also store the coordinates before and after mimimization for comparison.

In [7]:
minimizedPotentials = np.zeros(numFrames)
for i in tqdm(mt.frames(highPotentialFrames)):
    mt.pdbReporter('selected_pt.pdb')
    mt.minimizeEnergy(maxIterations = 100)
    minimizedPotentials[i] = mt.getPotentialEnergy()
    mt.pdbReporter('minimized_pt.pdb')

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:48<00:00,  4.81s/it]


mt.minimizeEnergy() passes its arguments directly to the openMM function of the same name. After minimization we simply extract the current potential energy of the box and store it in our array. Again, this is only done for a few frames to save time. If we wanted this for all frames in the trajectory, we could have these steps to the first loop where we evaluated the potential energy.

Now lets first see the differences in energies before and after minimization:

In [8]:
for i in range(numFrames):
    print('frame: {:=4} | pre: {:>-10.3e} | post: {:>-10.3e} | change: {:>-10.3e}'.format(highPotentialFrames[i], 
                                                         potentials[highPotentialFrames[i]], minimizedPotentials[i], 
                                                         minimizedPotentials[i]-potentials[highPotentialFrames[i]]))

frame: 2008 | pre:  5.969e+18 | post:  1.053e+06 | change: -5.969e+18
frame: 1809 | pre:  2.390e+16 | post:  5.711e+06 | change: -2.390e+16
frame: 1608 | pre:  1.075e+16 | post: -4.545e+03 | change: -1.075e+16
frame:  992 | pre:  4.532e+15 | post:  2.301e+05 | change: -4.532e+15
frame: 1843 | pre:  1.479e+15 | post:  8.041e+06 | change: -1.479e+15
frame: 1585 | pre:  1.028e+15 | post: -6.599e+03 | change: -1.028e+15
frame:    3 | pre:  5.796e+14 | post:  7.003e+11 | change: -5.789e+14
frame:  161 | pre:  1.099e+14 | post: -6.569e+03 | change: -1.099e+14
frame: 1617 | pre:  1.472e+13 | post:  6.934e+08 | change: -1.472e+13
frame: 1536 | pre:  1.117e+13 | post:  1.299e+07 | change: -1.117e+13


Now for a visual comparison:

In [9]:
uBefore = mda.Universe('selected_pt.pdb')
viewBefore = nv.show_mdanalysis(uBefore)
viewBefore.add_licorice('sidechain or .CA')
viewBefore.add_spacefill('not protein')
viewBefore

NGLWidget(max_frame=9)

In [10]:
uAfter = mda.Universe('minimized_pt.pdb')
viewAfter = nv.show_mdanalysis(uBefore)
viewAfter.add_licorice('sidechain or .CA')
viewAfter.add_spacefill('not protein')
viewAfter

NGLWidget(max_frame=9)